# README    
1. generate a Keras Layer :AttLayer  
2. Use csv module to import data


# Result  
1)使用Kmer进行输入 没有效果 
  predict 一下 有预测输出，说明训练有效果
  但是 无论是正负样本，预测成负样本的value远远大于正样本的（0.8》》0.2）
  可能是训练时正负样本悬殊引起的
  

In [1]:
import numpy as np
import pandas as pd
import sys
import os

os.environ['KERAS_BACKEND']='theano'
from keras.engine.topology import Layer
from keras import initializations
from keras import backend as K

from collections  import defaultdict
import pandas as pd
import numpy as np

Using Theano backend.
Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5105)


In [2]:
class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializations.get('normal')
        #self.input_spec = [InputSpec(ndim=3)]
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        #self.W = self.init((input_shape[-1],1))
        self.W = self.init((input_shape[-1],))
        #self.input_spec = [InputSpec(shape=input_shape)]
        self.trainable_weights = [self.W]
        super(AttLayer, self).build(input_shape)  # be sure you call this somewhere!

    def call(self, x, mask=None):


        M = K.tanh(x)
        alpha = K.dot(M,self.W)#.dimshuffle(0,2,1)

        ai = K.exp(alpha)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')
        weighted_input = x*weights.dimshuffle(0,1,'x')
        return K.tanh(weighted_input.sum(axis=1))
        '''
        eij = K.tanh(K.dot(x, self.W))

        ai = K.exp(eij)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')

        weighted_input = x*weights.dimshuffle(0,1,'x')
        return weighted_input.sum(axis=1)
        '''
    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[-1])


#Attention GRU network
class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializations.get('normal')
        #self.input_spec = [InputSpec(ndim=3)]
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        #self.W = self.init((input_shape[-1],1))
        self.W = self.init((input_shape[-1],))
        #self.input_spec = [InputSpec(shape=input_shape)]
        self.trainable_weights = [self.W]
        super(AttLayer, self).build(input_shape)  # be sure you call this somewhere!

    def call(self, x, mask=None):
        '''   
        M = K.tanh(x)
        alpha = K.dot(M,self.W)#.dimshuffle(0,2,1)

        ai = K.exp(alpha)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')
        weighted_input = x*weights.dimshuffle(0,1,'x')
        return K.tanh(weighted_input.sum(axis=1))
        '''
        eij = K.tanh(K.dot(x, self.W))

        ai = K.exp(eij)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')

        weighted_input = x*weights.dimshuffle(0,1,'x')
        return weighted_input.sum(axis=1)
        
    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[-1])

# Read Data

In [3]:
datafile  =  '/home/yinqijin/WorkSpace/2.RNA_Structure_Profile/Orig_data/pairs.csv'
import csv

with open(datafile) as csvfile:
    spamreader = csv.reader(csvfile)
    for  row in spamreader:
        csvkeys = row
        break
print csvkeys

csvdata = dict()
for item in csvkeys:
    csvdata[item] =[]
with open(datafile) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        # Add values of every keys to the csvdata for every row
        for item in csvkeys:
            csvdata[item].append(row[item])
            if abs(int(row['enhancer_start'] )-int( row['enhancer_end'])) <6:
                print row
    

In [6]:
#print csvdata.keys()
for i in range(1):
    print '-'*40
    for item in csvkeys:
        print item ,': ',csvdata[item][5000]

----------------------------------------
bin :  [22271, 83570.8]
enhancer_chrom :  chr22
enhancer_distance_to_promoter :  32514
enhancer_end :  43515000
enhancer_name :  GM12878|chr22:43514400-43515000
enhancer_start :  43514400
label :  0
promoter_chrom :  chr22
promoter_end :  43548957
promoter_name :  GM12878|chr22:43547516-43548957
promoter_start :  43547516
window_end :  43547515
window_start :  43515001
window_chrom :  chr22
window_name :  GM12878|chr22:43515001-43547515
interactions_in_window :  0
active_promoters_in_window :  1


## Make Promoter/Enhancer Sequence

In [7]:
from pyfasta import Fasta
genome = Fasta('/home/yinqijin/WorkSpace/DataHub/genome.fa')

In [8]:
#证明序列不会有错
import numpy as np
chrom = np.unique(csvdata['enhancer_chrom'])
print chrom
for item in chrom:
    #print genome[item][100:102]

IndentationError: expected an indented block (<ipython-input-8-66f86077914a>, line 6)

In [10]:
#获得ｋｍｅｒ的函数
d2 = {'a':0, 'A':0, 'g':1, 'G':1, 'c':2, 'C':2, 't':3, 'T':3, 'N':0, 'n':0}
def seq_to_kspec(seq):
    mat = np.zeros((4096,1))
    k = 0
    if len(seq)<6:
        return mat
    for i in range(6):
        k = k*4 + d2[seq[i]]
    mat[k]+=1  
    for i in range(6,len(seq)):
        k = k - 4**5*d2[seq[i-6]]
        k = k*4 + d2[seq[i]]
        mat[k] += 1  
    return mat

In [11]:
PESeq=dict()
PESeq['Pro-index']=[]
PESeq['Pro-Seq']=[]
PESeq['Pro-Kmer']=[]
PESeq['Ehr-index']=[]
PESeq['Ehr-Seq']=[]
PESeq['Ehr-Kmer']=[]
PESeq['label'] =[]

In [12]:
for index in range( len( csvdata[csvkeys[0]])):
    pro_index = [csvdata['promoter_chrom'][index],csvdata['promoter_start'][index],csvdata['promoter_end'][index]]   
    ehr_index = [csvdata['enhancer_chrom'][index],csvdata['enhancer_start'][index] ,csvdata['enhancer_end'][index]]
    pro_seq =  genome[csvdata['promoter_chrom'][index]][int(csvdata['promoter_start'][index]) :int(csvdata['promoter_end'][index])].upper()
    ehr_seq =  genome[csvdata['enhancer_chrom'][index]] [int(csvdata['enhancer_start'][index]) : int(csvdata['enhancer_end'][index])].upper()    

    if len(pro_seq)<6:
        print index,'\tpro_index',pro_index
        continue
    if len(ehr_seq)<6:
        print index,'\tehr_index',ehr_index
        continue
    PESeq['Pro-index'].append(pro_index )
    PESeq['Pro-Seq'].append(pro_seq)
    PESeq['Pro-Kmer'].append( seq_to_kspec(pro_seq))
    PESeq['Ehr-index'].append(ehr_index)
    PESeq['Ehr-Seq'].append(ehr_seq)
    PESeq['Ehr-Kmer'].append(seq_to_kspec(ehr_seq))
    PESeq['label'].append(csvdata['label'][index])

1452 	ehr_index ['chr17', '41080595', '41080600']
1760 	ehr_index ['chr3', '144015595', '144015600']
1887 	ehr_index ['chr6', '27857195', '27857200']
2937 	ehr_index ['chr1', '149764795', '149764800']
2943 	ehr_index ['chr20', '44642400', '44642405']
3065 	ehr_index ['chr6', '31793795', '31793800']
4066 	ehr_index ['chr11', '62541195', '62541200']
4186 	ehr_index ['chr19', '17576795', '17576800']
4510 	ehr_index ['chr10', '70758000', '70758005']
4598 	ehr_index ['chr19', '17829600', '17829605']
4866 	ehr_index ['chr9', '131801000', '131801005']
5335 	ehr_index ['chr3', '61169000', '61169005']
7503 	ehr_index ['chr14', '23285600', '23285605']
8441 	ehr_index ['chrX', '153725595', '153725600']
9221 	ehr_index ['chr4', '40580595', '40580600']
9249 	ehr_index ['chr11', '62541195', '62541200']
9430 	ehr_index ['chr4', '36313000', '36313005']
9445 	ehr_index ['chr14', '23285600', '23285605']
10193 	ehr_index ['chr12', '111135595', '111135600']
10649 	ehr_index ['chr4', '159783200', '15978320

In [13]:
print '-'*50
for item in PESeq.keys():
    print item, np.shape(PESeq[item])
print 'pos data' , sum(int( item ) for item in PESeq['label'])
print 'neg data',sum(int( 1) if item =='0' else int(0) for item in PESeq['label'])



--------------------------------------------------
Ehr-Kmer (44236, 4096, 1)
Ehr-Seq (44236,)
Pro-index (44236, 3)
Pro-Kmer (44236, 4096, 1)
Pro-Seq (44236,)
Ehr-index (44236, 3)
label (44236,)
pos data 2110
neg data 42126


## cut negative samples

In [14]:
pos_neg_index = [ int(item) for item in PESeq['label']]
print pos_neg_index[:50]
pos_neg_index = np.array(pos_neg_index )  #很关键

neg_index  = np.where(pos_neg_index==0)
pos_index  = np.where(pos_neg_index ==1)
print neg_index
print pos_index
bal_pos_neg_index = pos_index + neg_index[:len(pos_index)]
print bal_pos_neg_index[:50]
print bal_pos_neg_index[-50:]
print len(bal_pos_neg_index)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
(array([ 2110,  2111,  2112, ..., 44233, 44234, 44235]),)
(array([   0,    1,    2, ..., 2107, 2108, 2109]),)
(array([   0,    1,    2, ..., 2107, 2108, 2109]), array([ 2110,  2111,  2112, ..., 44233, 44234, 44235]))
(array([   0,    1,    2, ..., 2107, 2108, 2109]), array([ 2110,  2111,  2112, ..., 44233, 44234, 44235]))
2


In [15]:
print 'lengthest Enhancer',len(max(PESeq['Ehr-Seq'],key= lambda x:len(x)))
print  'lengthest Promoter',len(max(PESeq['Pro-Seq'],key= lambda x:len(x)))

lengthest Enhancer 9466
lengthest Promoter 18594


#Get X,Y with no  limited
X=[]
Y=[]
for index in range( len( PESeq['label'])):
    Y.append(PESeq['label'][index])
    X.append(np.append(PESeq['Pro-Kmer'][index],PESeq['Ehr-Kmer'][index],axis=1))
print np.shape(X)
print np.shape(Y)
X = np.array(X)
Y  = np.array(Y)

index = range(len(PESeq['label']))

In [16]:
# Get X,Y with   limite  making positive and negative samples 's numbers is the same
X=[]
Y=[]
for index in range( 4220):
    Y.append(PESeq['label'][index])
    X.append(np.append(PESeq['Pro-Kmer'][index],PESeq['Ehr-Kmer'][index],axis=1))
print np.shape(X)
print np.shape(Y)
X = np.array(X)
Y  = np.array(Y)


index = range(4220)

(4220, 4096, 2)
(4220,)


In [17]:
from keras.utils.np_utils import to_categorical

In [18]:

VALIDATION_SPLIT =0.2

np.random.seed(1234)

np.random.shuffle(index)
nb_validation_samples = int(VALIDATION_SPLIT*len(Y))

In [19]:

x_train = X[index[: - nb_validation_samples]]
y_train = Y[index[:-nb_validation_samples]]
x_val = X[index[-nb_validation_samples:]]
y_val =Y[index[-nb_validation_samples:]]



In [20]:
y_train = to_categorical(y_train) #one-hot 编码labels
y_val = to_categorical(np.asarray(y_val))

In [21]:
print 'x_train_shape:',np.shape(x_train)
print 'y_train_shape:',np.shape(y_train)

x_train_shape: (3376, 4096, 2)
y_train_shape: (3376, 2)


#使验证集正负样本数量相等
print np.shape(y_val)

print 'valid负正样本比例' sum(y_val)


count = defaultdict()
count['0']=[]
count['1']=[]
for i in range(8847):
    if y_val[i][0]==1:
        count['0'].append(i)
    else:
        count['1'].append(i)
print len(count['0']),len(count['1'])


 index_my = count['0'][:401]+count['1']

x_val_my = x_val[index_my,:,:]
y_val_my  =y_val[index_my,:]

print 'x_val_shape:',np.shape(x_val_my)
print 'y_val_shape:',np.shape(y_val_my)

In [22]:
print y_val_my
print x_val_my

NameError: name 'y_val_my' is not defined

# Make Network

In [22]:
from keras.layers import Embedding
from keras.layers import Dense,Input,Activation
from keras.layers import Embedding, LSTM, Bidirectional,GRU,InputLayer
from keras.models import Model,Sequential
from  keras.regularizers import ActivityRegularizer
from keras.layers.core import Dropout

In [23]:
from keras import layers

In [44]:
kmer_input = Input(shape=(4096,2), dtype='float32')

In [45]:
l_lstm =Bidirectional(LSTM(2,return_sequences=False))(kmer_input)

In [48]:
l_lstm_drop = Dropout(0.3)(l_lstm)
#l_att = AttLayer()(l_lstm_drop)
#l_att_drop = Dropout(0.5)(l_att)
preds = Dense(len( y_train[0]), activation='softmax',activity_regularizer= ActivityRegularizer(l2=0.005))(l_lstm_drop)
model  = Model (kmer_input,preds)

In [49]:
model.compile(loss='mse',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - attention LSTM network")
model.summary()


model fitting - attention LSTM network
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_5 (InputLayer)             (None, 4096, 2)       0                                            
____________________________________________________________________________________________________
bidirectional_5 (Bidirectional)  (None, 4)             80          input_5[0][0]                    
____________________________________________________________________________________________________
dropout_11 (Dropout)             (None, 4)             0           bidirectional_5[0][0]            
____________________________________________________________________________________________________
dense_6 (Dense)                  (None, 2)             10          dropout_11[0][0]                 
Total params: 90
Trainable params: 90
Non-trainable 

In [50]:
model.fit(x_train, y_train, validation_data=(x_val, y_val), nb_epoch=20, batch_size=100)

Train on 3376 samples, validate on 844 samples
Epoch 1/20
3376/3376 [==============================] - 545s - loss: 0.5085 - acc: 0.5409 - val_loss: 0.4942 - val_acc: 0.5427
Epoch 2/20
3376/3376 [==============================] - 545s - loss: 0.5048 - acc: 0.5391 - val_loss: 0.4920 - val_acc: 0.5592
Epoch 3/20
3376/3376 [==============================] - 548s - loss: 0.5031 - acc: 0.5435 - val_loss: 0.4904 - val_acc: 0.5628
Epoch 4/20
3376/3376 [==============================] - 551s - loss: 0.4997 - acc: 0.5616 - val_loss: 0.4895 - val_acc: 0.5735
Epoch 5/20
3376/3376 [==============================] - 546s - loss: 0.4984 - acc: 0.5501 - val_loss: 0.4889 - val_acc: 0.5723
Epoch 6/20
3376/3376 [==============================] - 544s - loss: 0.4986 - acc: 0.5533 - val_loss: 0.4883 - val_acc: 0.5770
Epoch 7/20
3376/3376 [==============================] - 538s - loss: 0.4976 - acc: 0.5554 - val_loss: 0.4879 - val_acc: 0.5794
Epoch 8/20
3376/3376 [==============================] - 550s - l

In [51]:
model.fit(x_train, y_train, validation_data=(x_val, y_val), nb_epoch=20, batch_size=100)

Train on 3376 samples, validate on 844 samples
Epoch 1/20
3376/3376 [==============================] - 548s - loss: 0.4932 - acc: 0.5749 - val_loss: 0.4860 - val_acc: 0.6209
Epoch 2/20
3376/3376 [==============================] - 555s - loss: 0.4935 - acc: 0.5773 - val_loss: 0.4858 - val_acc: 0.6232
Epoch 3/20
3376/3376 [==============================] - 549s - loss: 0.4928 - acc: 0.5809 - val_loss: 0.4856 - val_acc: 0.6220
Epoch 4/20
3376/3376 [==============================] - 557s - loss: 0.4934 - acc: 0.5749 - val_loss: 0.4857 - val_acc: 0.6256
Epoch 5/20
3376/3376 [==============================] - 555s - loss: 0.4934 - acc: 0.5749 - val_loss: 0.4858 - val_acc: 0.6232
Epoch 6/20
3376/3376 [==============================] - 552s - loss: 0.4932 - acc: 0.5868 - val_loss: 0.4857 - val_acc: 0.6232
Epoch 7/20
3376/3376 [==============================] - 548s - loss: 0.4935 - acc: 0.5776 - val_loss: 0.4857 - val_acc: 0.6244
Epoch 8/20
3376/3376 [==============================] - 544s - l

In [52]:
model.fit(x_train, y_train, validation_data=(x_val, y_val), nb_epoch=50, batch_size=100)

Train on 3376 samples, validate on 844 samples
Epoch 1/50
3376/3376 [==============================] - 563s - loss: 0.4931 - acc: 0.5770 - val_loss: 0.4855 - val_acc: 0.6197
Epoch 2/50
3376/3376 [==============================] - 564s - loss: 0.4922 - acc: 0.5764 - val_loss: 0.4854 - val_acc: 0.6232
Epoch 3/50
3376/3376 [==============================] - 562s - loss: 0.4929 - acc: 0.5800 - val_loss: 0.4855 - val_acc: 0.6173
Epoch 4/50
3376/3376 [==============================] - 563s - loss: 0.4931 - acc: 0.5741 - val_loss: 0.4855 - val_acc: 0.6197
Epoch 5/50
3376/3376 [==============================] - 551s - loss: 0.4923 - acc: 0.5859 - val_loss: 0.4854 - val_acc: 0.6173
Epoch 6/50
3376/3376 [==============================] - 564s - loss: 0.4927 - acc: 0.5829 - val_loss: 0.4853 - val_acc: 0.6197
Epoch 7/50
3376/3376 [==============================] - 563s - loss: 0.4936 - acc: 0.5714 - val_loss: 0.4855 - val_acc: 0.6161
Epoch 8/50
3376/3376 [==============================] - 563s - l

In [ ]:
pro = model.predict_on_batch(x_val)

In [ ]:
print pro

# Save&Search

In [67]:
dir(model)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_collected_trainable_weights',
 '_fit_loop',
 '_function_kwargs',
 '_get_node_attribute_at_index',
 '_make_predict_function',
 '_make_test_function',
 '_make_train_function',
 '_output_mask_cache',
 '_output_shape_cache',
 '_output_tensor_cache',
 '_predict_loop',
 '_standardize_user_data',
 '_test_loop',
 '_updated_config',
 'add_inbound_node',
 'add_loss',
 'add_update',
 'add_weight',
 'assert_input_compatibility',
 'build',
 'built',
 'call',
 'compile',
 'compute_mask',
 'constraints',
 'container_nodes',
 'count_params',
 'create_input_layer',
 'evaluate',
 'evaluate_generator',
 'fit',
 'fit_generator',
 'from_config',
 'get_config',
 'get_input_at',
 'get_input_mask_at',
 'get_input_shape_at',
 'ge

In [69]:
model.__dict__

{'_collected_trainable_weights': [<CudaNdarrayType(float32, vector)>,
  backward_lstm_1_U_c,
  backward_lstm_1_U_f,
  backward_lstm_1_U_i,
  backward_lstm_1_U_o,
  backward_lstm_1_W_c,
  backward_lstm_1_W_f,
  backward_lstm_1_W_i,
  backward_lstm_1_W_o,
  backward_lstm_1_b_c,
  backward_lstm_1_b_f,
  backward_lstm_1_b_i,
  backward_lstm_1_b_o,
  dense_1_W,
  dense_1_b,
  forward_lstm_1_U_c,
  forward_lstm_1_U_f,
  forward_lstm_1_U_i,
  forward_lstm_1_U_o,
  forward_lstm_1_W_c,
  forward_lstm_1_W_f,
  forward_lstm_1_W_i,
  forward_lstm_1_W_o,
  forward_lstm_1_b_c,
  forward_lstm_1_b_f,
  forward_lstm_1_b_i,
  forward_lstm_1_b_o],
 '_function_kwargs': {},
 '_output_mask_cache': {'140689816233104_140699161811792': None},
 '_output_shape_cache': {},
 '_output_tensor_cache': {},
 'built': True,
 'container_nodes': {'attlayer_1_ib-0',
  'bidirectional_1_ib-0',
  'dense_1_ib-0',
  'dropout_1_ib-0',
  'input_1_ib-0'},
 'history': <keras.callbacks.History at 0x7ff3d07675d0>,
 'inbound_nodes': [

In [54]:
model.save_weights('./Gen_data/10_Att-BLSTM.h5')

In [29]:
model.load_weights('./Gen_data/10_Att-BLSTM.h5',by_name=True)